In [1]:
include("../../src/Yggdrasil.jl")
using .Yggdrasil

In [9]:
data_folder = "../example_data/test_01/"

#-------------- get meta information on initial snapshot --------------
snap = 101                                        # Number of the snapshot
initial_snap = read_snapshot(data_folder, snap)   # Load initial snapshot meta
IDX = initial_snap.IDX                            # Index information
system = initial_snap.SYSTEM;                     # Unit system
#-----------------------------------------------------------------------------

#--------------- define which snapshot to load ---------------
i_start = 101   # Start snapshot index
i_end  = 110    # End snapshot index
stride = 1      # Stride - how many snapshots to skip between loads
n_snaps = Int(floor( (i_end - i_start) / stride) + 1)  # total number of snapshots to load
load_pic = false; # If true, will only load PIC patch data

# Example desription - load multiple snapshot
Below is shown first how to load multiple snapshots into a data and times array. 
This is the simplest way to load multiple snapshots. For 1D and 2D experiment, using the "drop_units_dims" removes the collapsed dimensions from the array. 

Next is shown how to get the cartesian coordinates for the snapshot data.

The function works by first inspecting the meta information in the snapshot. From this a global array to include all data is constructed. It is assumed that patches are in a cartesian geometry

In [ ]:
#---------------------- simplest way to load snapshots ----------------------
data, times = load_multiple_snapshots(data_folder,             # folder input 
                                      i_start, i_end, stride,  # snapshot range
                                      load_pic)                # load PIC or MHD
data = drop_unit_dims(data); # Drop array dimension with size 1. 
# NOTE if only 1 snapshot is loadded it will also collapse the "time" dimension

In [ ]:
drop_dims = true  # drops array dimension with size 1 in the cartesian coordinates
x,y,z,ds = get_xyz(initial_snap, drop_dims);         # Get the x, y, z coordinates and dataset from the initial snapshot

# Example desription - load multiple snapshot in area
Below is shown how to load only an area of a snapshot, and can be used to fetch information at different resolutions. 

It requires first the definition of an area, which can most easily be done using the "get_area" function. This function calculates the lower left corner (LLC) and upper right corner (URC) based on a midpoint, a number of patches, and the level to load. The level is set by the "ref_level" constant. 

The function calculates the LLC and URC in integer indices based on the distance to the simulation LLC, and with the cell size defined by the level. It aligns the LLC and URC so they are located with the nearest patch border. 

This version is useful to optimize load time and memory usage, especially when working with AMR setups, or large simulations. 


Similar to the simple case, we can load the cartesian x,y,z coordinates using the "get_xyz" function

In [ ]:
#------------ Get area defined by LLC and URC aligned with patches --------------   
    #------------ set refinement level -------------------
    ref_level = 3  # zero meants no refiment (coarsest grid level)
    #-----------------------------------------------------
    #----------- set level based on refinement level -----------
    level_min = initial_snap.LEVELMIN
    level = level_min + ref_level 
    #-----------------------------------------------------
    #------------- define midpoint of area and number of patches
    midpoint = [16.27207,  0.0, 6.1138]
    patches = 4
    #-----------------------------------------------------
llc, urc, llc_pos, urc_pos  = get_area(initial_snap, midpoint, patches, level);

In [ ]:
#------------ Get area defined by LLC and URC aligned with patches --------------   
    #------------ set refinement level -------------------
    ref_level = 3  # zero meants no refiment (coarsest grid level)
    #-----------------------------------------------------
    #----------- set level based on refinement level -----------
    level_min = initial_snap.LEVELMIN
    level = level_min + ref_level 
    #-----------------------------------------------------
    #------------- define midpoint of area and number of patches
    midpoint = [16.27207,  0.0, 6.1138]
    patches = 4
    #-----------------------------------------------------
llc, urc, llc_pos, urc_pos  = get_area(initial_snap, midpoint, patches, level);
#---------------------------------------------------------------------------------
#---------------------- load snapshots in the area ----------------------
#--------- first get cartesian coordinates for the area -----------
drop_dims = true  # drops array dimension with size 1 in the cartesian coordinates
    #--------- in example 2D case, y dimension is collapse, so we can ignore it like this 
x,_,z,ds = get_xyz(initial_snap, true, level; llc=llc, urc=urc);
#-------------------------------------------------------------------
    # optional get 1D coordinates for x and z
x_vals = x[:, 1]
z_vals = z[1, :];
#-------------------------------------------------------------------

#---------------- load snapshot data in the area ----------------------
data, times = load_multiple_snapshots(data_folder, 
                                      i_start, i_end, stride, 
                                      llc, urc,
                                      load_pic; 
                                      use_level=level)
data = drop_unit_dims(data);
#-------------------------------------------------------------------
